## 딥러닝 과제
1. 11주차에 사용된 인물사진 이미지 데이터를 사용해 생성모델을 활용하여 가짜 인물 이미지를 생성하는 코드를 완성하여라.
2. 1번에서 완성한 코드를 실행하여 도출된 가짜를 2~3개를 제출하여라.<br>
최종적으로 1번 코드와 2번 이미지 결과를 각각 첨부하여 제출하여라.

In [1]:
import math
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.utils as vutils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
data_root = '/content/drive/MyDrive/2025_1_Colab/DeepLearning/Lib/SRGAN'

In [3]:
nz = 100
ngf = 128
ndf = 128
nc = 3
img_size = 64
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.CenterCrop(img_size),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 512

train_dataset = datasets.ImageFolder(
    root = data_root,
    transform = transform
)

train_loader = DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
)

In [4]:
# IMGSIZE 64일때 실행
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc, img_size): # img_size는 참고용일 수 있음
        super(Generator, self).__init__()
        self.nz = nz
        self.ngf = ngf
        self.nc = nc
        # self.img_size = img_size # 직접적인 레이어 크기 결정에는 사용 안 할 수도 있음

        self.main = nn.Sequential(
            # 입력: nz (잠재 벡터)
            # 첫 번째 ConvTranspose2d: nz를 (ngf * 8) 채널의 4x4 특징 맵으로 변환
            # 출력: (ngf * 8) x 4 x 4
            nn.ConvTranspose2d(nz, ngf * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # 두 번째 ConvTranspose2d: 4x4 -> 8x8
            # 입력: (ngf * 8) x 4 x 4
            # 출력: (ngf * 4) x 8 x 8
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            # 세 번째 ConvTranspose2d: 8x8 -> 16x16
            # 입력: (ngf * 4) x 8 x 8
            # 출력: (ngf * 2) x 16 x 16
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            # 네 번째 ConvTranspose2d: 16x16 -> 32x32
            # 입력: (ngf * 2) x 16 x 16
            # 출력: ngf x 32 x 32
            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # 다섯 번째 ConvTranspose2d: 32x32 -> 64x64 (최종 이미지 크기)
            # 입력: ngf x 32 x 32
            # 출력: nc x 64 x 64
            nn.ConvTranspose2d(ngf, nc, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()  # 출력 픽셀 값을 [-1, 1] 범위로
        )

    def forward(self, input):
        # 입력 input은 (batch_size, nz) 형태의 2D 텐서
        # 이를 (batch_size, nz, 1, 1) 형태의 4D 텐서로 변환하여 ConvTranspose2d에 전달
        return self.main(input.view(-1, self.nz, 1, 1))
class Discriminator(nn.Module):
    def __init__(self, nc, ndf, img_size): # img_size는 참고용일 수 있음
        super(Discriminator, self).__init__()
        self.nc = nc
        self.ndf = ndf
        # self.img_size = img_size # 직접적인 레이어 크기 결정에는 사용 안 할 수도 있음

        self.main = nn.Sequential(
            # 입력: nc x 64 x 64
            # 첫 번째 Conv2d: 64x64 -> 32x32
            # 출력: ndf x 32 x 32
            nn.Conv2d(nc, ndf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True), # 첫 레이어에는 BatchNorm 보통 생략

            # 두 번째 Conv2d: 32x32 -> 16x16
            # 입력: ndf x 32 x 32
            # 출력: (ndf * 2) x 16 x 16
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # 세 번째 Conv2d: 16x16 -> 8x8
            # 입력: (ndf * 2) x 16 x 16
            # 출력: (ndf * 4) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # 네 번째 Conv2d: 8x8 -> 4x4
            # 입력: (ndf * 4) x 8 x 8
            # 출력: (ndf * 8) x 4 x 4
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            # 다섯 번째 Conv2d: 4x4 특징맵을 1x1 스칼라 값으로
            # 입력: (ndf * 8) x 4 x 4
            # 출력: 1 x 1 x 1
            nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()  # 출력을 0과 1 사이의 확률 값으로
        )

    def forward(self, input):
        # 입력 input은 (batch_size, nc, 64, 64) 형태의 이미지 텐서
        # 최종 출력은 (batch_size, 1) 형태가 되도록 view 조정
        return self.main(input).view(-1, 1)

In [ ]:
# IMGSIZE 32일때 실행
class Generator(nn.Module):
    def __init__(self, nz, ngf ,nc, img_size):
        super(Generator, self).__init__()
        self.nz = nz
        self.ngf = ngf
        self.nc = nc
        self.img_size = img_size

        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 4, kernel_size = 4, stride = 1, padding = 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        return self.main(input.view(-1, self.nz, 1, 1))

class Discriminator(nn.Module):
    def __init__(self, nc, ndf, img_size): # nc, img_size 추가
        super(Discriminator, self).__init__()
        self.nc = nc
        self.img_size = img_size
        self.ndf = ndf
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, kernel_size = 4, stride = 2, padding = 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, 1, kernel_size = 4, stride = 1, padding = 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
      return self.main(input).view(-1, 1)

In [5]:
generator = Generator(nz, ngf, nc, img_size).to(device) # nc, image_sz 전달
discriminator = Discriminator(nc, ndf, img_size).to(device) # nc, image_sz 전달
print(generator)
print(discriminator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()


In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [7]:
generator.apply(weights_init)
discriminator.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [8]:
optim_g = optim.Adam(generator.parameters(), lr=0.001, betas=(0.5, 0.999))
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.MSELoss()

losses_g = [] # 생성자 오차 저장
losses_d = [] # 판별자 오차 저장
images = []

In [9]:
def train_discriminator(optimizer, data_real, data_fake_detached, criterion, device):
    b_size = data_real.size(0)

    smooth_real = 0.9
    smooth_fake = 0.0

    real_label = torch.full((b_size, 1), smooth_real, dtype=torch.float, device = device)
    fake_label = torch.zeros(b_size, 1, dtype=torch.float, device=device)

    optimizer.zero_grad()

    output_real = discriminator(data_real)
    loss_real = criterion(output_real, real_label)

    output_fake = discriminator(data_fake_detached)
    loss_fake = criterion(output_fake, fake_label)

    loss_total = (loss_real + loss_fake)/2
    loss_total.backward()
    optimizer.step()
    return loss_total.item()

In [10]:
def train_generator(optimizer, data_fake, criterion, device):
    b_size = data_fake.size(0)
    real_label = torch.ones(b_size, 1).to(device)

    optimizer.zero_grad()
    output = discriminator(data_fake)

    loss = criterion(output, real_label)
    loss.backward()
    optimizer.step()
    return loss.item()

In [12]:
num_samples_to_show = 16 # 4x4 그리드로 표시할 이미지 수
fixed_noise = torch.randn(num_samples_to_show, nz, 1, 1, device=device) # DCGAN 입력 형태

epochs = 1000
for epoch in range(epochs):
    loss_g_sum = 0.0
    loss_d_sum = 0.0
    num_batches = 0 # 변수명 수정: num_batchs -> num_batches

    generator.train() # 학습 모드 설정
    discriminator.train() # 학습 모드 설정

    for idx, data in enumerate(train_loader):
        image, _ = data
        data_real_batch = image.to(device)
        current_batch_size = data_real_batch.size(0)

        if current_batch_size == 0: continue

        noise_for_d = torch.randn(current_batch_size, nz, 1, 1, device=device) # FUCK
        data_fake_for_d = generator(noise_for_d).detach()
        loss_d_batch = train_discriminator(optim_d, data_real_batch, data_fake_for_d, criterion, device)
        loss_d_sum += loss_d_batch

        noise_for_g = torch.randn(current_batch_size, nz, 1, 1, device=device) # DCGAN 생성자 입력 형태
        data_fake_for_g = generator(noise_for_g)
        loss_g_batch = train_generator(optim_g, data_fake_for_g, criterion, device)
        loss_g_sum += loss_g_batch

        num_batches += 1 # 여기서 num_batches 증가

    if num_batches > 0:
        avg_epoch_loss_g = loss_g_sum / num_batches
        avg_epoch_loss_d = loss_d_sum / num_batches
    else:
        avg_epoch_loss_g = 0
        avg_epoch_loss_d = 0
        print(f"경고: 에포크 {epoch}에서 처리된 배치가 없습니다.")

    losses_g.append(avg_epoch_loss_g) # 리스트에 추가
    losses_d.append(avg_epoch_loss_d) # 리스트에 추가

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}") # epochs는 총 에포크 수
        print(f"Avg Generator loss: {avg_epoch_loss_g:.4f}, Avg Discriminator loss: {avg_epoch_loss_d:.4f}")

    if (epoch + 1) % 100 == 0 or epoch == epochs - 1: # 첫 에포크, 매 100 에포크, 마지막 에포크
        # 생성된 이미지 시각화
        generator.eval() # 평가 모드로 전환
        with torch.no_grad(): # 그래디언트 계산 비활성화
            fake_images_sample = generator(fixed_noise).detach().cpu()

            # Matplotlib으로 플롯
            grid_img = vutils.make_grid(fake_images_sample, nrow=int(math.sqrt(num_samples_to_show)), padding=2, normalize=True)
            plt.figure(figsize=(8, 8)) # 그림 크기
            plt.axis("off")
            plt.title(f"Generated Fake Images at Epoch {epoch+1}")
            plt.imshow(np.transpose(grid_img, (1, 2, 0))) # C, H, W -> H, W, C

            plt.show() # Colab 등에서 바로 이미지 보기

        generator.train() # 다시 학습 모드로 전환

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plt.figure()
plt.plot(torch.Tensor(losses_g), label = 'Generator loss')
plt.plot(torch.Tensor(losses_d), label = 'Discriminator Loss')
plt.legend()

In [ ]:
num_images_to_save = 2  # 저장할 가짜 이미지의 개수
output_dir = 'individual_fake_images' # 이미지를 저장할 폴더 이름
os.makedirs(output_dir, exist_ok=True) # 폴더가 없으면 생성

# 모델을 평가 모드로 설정
generator.eval()

# 그래디언트 계산 비활성화
with torch.no_grad():
    # 저장할 개수만큼의 노이즈 생성
    # 각 이미지는 서로 다른 노이즈로부터 생성되도록 함
    noise = torch.randn(num_images_to_save, nz, 1, 1, device=device) # DCGAN 입력 형태

    # 생성자로부터 가짜 이미지 생성
    fake_images_batch = generator(noise).detach().cpu() # (num_images_to_save, nc, img_size, img_size) 형태

    # --- 생성된 가짜 이미지를 개별 파일로 저장 ---
    for i in range(num_images_to_save):
        # fake_images_batch[i]는 단일 이미지 텐서 (nc, img_size, img_size)
        # PyTorch의 save_image는 (C, H, W) 또는 (B, C, H, W) 형태를 받으므로,
        # 단일 이미지를 저장할 때는 (C, H, W)로 전달하거나,
        # 배치 차원을 유지한 채 (1, C, H, W)로 전달하고 nrow=1로 설정할 수 있습니다.
        # 여기서는 간단하게 단일 이미지 텐서를 직접 전달합니다.

        image_tensor_to_save = fake_images_batch[i] # (nc, img_size, img_size)

        # 파일 경로 설정 (예: fake_image_1.png, fake_image_2.png)
        file_path = os.path.join(output_dir, f"fake_image_{i+1}.png")

        # 단일 이미지 저장
        # normalize=True: 픽셀 값을 [0, 1] 범위로 정규화 (Tanh 출력 [-1,1]을 고려)
        vutils.save_image(image_tensor_to_save, file_path, normalize=True)

        print(f"가짜 이미지 {i+1} 저장 완료: {file_path}")


# (선택 사항) 저장된 이미지 중 하나를 Matplotlib으로 바로 확인하고 싶다면:
# if num_images_to_save > 0:
#     # 첫 번째로 저장된 이미지 (또는 원하는 이미지)를 불러와서 표시
#     # (위에서 fake_images_batch[0]을 사용해도 됨)
#     img_to_show = fake_images_batch[0] # 이미 CPU에 있는 상태
#     plt.figure(figsize=(4, 4)) # 그림 크기
#     plt.axis("off")
#     plt.title("One of the Generated Fake Images")
#     plt.imshow(np.transpose(img_to_show.numpy(), (1, 2, 0))) # C, H, W -> H, W, C
#     plt.show()

print(f"\n총 {num_images_to_save}개의 가짜 이미지가 '{output_dir}' 폴더에 저장되었습니다.")

가짜 이미지 1 저장 완료: individual_fake_images/fake_image_1.png
가짜 이미지 2 저장 완료: individual_fake_images/fake_image_2.png

총 2개의 가짜 이미지가 'individual_fake_images' 폴더에 저장되었습니다.
